# Geocoding Police Fire Departments and Mapping to CSI360 Customer Locations

- Import and data from CSI360 Police and Fire List.
- Geocode addresses to obtain latitude and longitude coordinates.
- Insert latitude and longitude columns into data file.
- Check each department location and identify if it is within 100 miles of the customer locations.
- Separate the dataframe for departments within 100 miles for each customer location.
- Export the data to CSV files for further analysis and reporting.

In [1]:
# Dependencies and Setup
import pandas as pd
from geopy.geocoders import GoogleV3
from geopy.distance import geodesic
# import folium # Use to create html map
# from folium import plugins # Use to create html map
from ratelimit import limits, sleep_and_retry
import time
import numpy as np # Use for Haversine formula

# Manually load API key from .env file. # Remove code associated with dotenv if this works
with open('C:/Users/jchan/csi360_fire_police/lefd-custy-targets/.env', 'r') as f:
    for line in f:
        if line.startswith('GOOGLE_MAPS_API_KEY'):
            google_api_key = line.split('=')[1].strip()

In [2]:
# Store filepath in a variable
# lefd_1_data = 'resources/data/lefd_1_data.csv' # completed filtering 1,000 records, so commented out
# Remove all instances of lefd_1_data and replace with lefd_csi_states to run 1,700+ records

lefd3 = 'resources/data/lefd_csi_states.csv'
csi360_customers = 'resources/data/csi360_customers.csv'

# Read each of the respective files (police, fire, agency_n, agency_addrs) and store into Pandas dataframe
lefd3_df = pd.read_csv(lefd3)
csi360_customers_df = pd.read_csv(csi360_customers)

# Convert the 'zip' column to integer, then back to string to remove decimals
csi360_customers_df['zip'] = csi360_customers_df['zip'].fillna("").apply(lambda x: str(int(float(x))) if x != "" else "")

display(lefd3_df.head(), csi360_customers_df.head())

,agency_name,agency_type,sworn_active_persnl,hq_addr1,hq_addr2,hq_city,hq_state,hq_zip,addr1,addr2,po_box,city,state,county,zip,hq_ph,hq_fax,org_type,website,fips
0,San Antonio Park Rangers,Special jurisdiction,112,,,,,,600 Hemisfair Park No. 337,,,San Antonio,TX,Bexar,78205,,,,,48029
1,College Station Police Department,Local police department,112,,,,,,2611 Texas Avenue,,,College Station,TX,Brazos,77842,,,,,48041
2,Kingsport Police Department,Local police department,111,,,,,,200 Shelby St.,,,Kingsport,TN,Sullivan,37660,,,,,47163
3,Dorchester County Sheriff's Office,Sheriff's office,111,,,,,,100 Sears St,,,Saint George,SC,Dorchester,29477,,,,,45035
4,Edinburg Police Department,Local police department,110,,,,,,100 E. Freddy Gonzalez Drive,1702 S. CLOSNER BLVD,,Edinburg,TX,Hidalgo,78539,,,,,48215


,company,contact,state,Last Contact,Status,zip
0,Berkeley County Sheriff's Office,Lt. Geno Alterio,SC,1/17/2023,Custy-5/20,29461
1,Blytheville Police Department,Vanessa Stewart,AZ,10/27/2023,Custy-10/20,72315
2,FEMA - Idaho,Keith Richey,NaN,NaN,NaN,
3,Franklin County Sheriff's Office,Rhonda Coyne,NC,8/22/2024,Custy-3/24,27549
4,Grant Parish Sheriff's Office,Cade Fletcher,LA,2/1/2023,Custy-8/22,71417


In [3]:
## Geocode CSI360 customer zip codes

In [4]:
# Initialize geolocator. 
geolocator = GoogleV3(api_key=google_api_key)

In [5]:
# Function to get geocode based on zip code

def get_lat_long(zip_code):
    if zip_code == "":  # Skip empty ZIP codes
        return None, None
    try:
        # Use the ZIP code directly to get the location
        location = geolocator.geocode(zip_code)
        if location:
            return location.latitude, location.longitude
        else:
            print(f"Geocoding failed for ZIP code: {zip_code}")  # Log failed geocodes
            return None, None
    except Exception as e:
        print(f"Error geocoding {zip_code}: {e}")
        return None, None

# Apply the geocode function to get Latitude and Longitude for each row in the cleaned DataFrame
csi360_customers_df['Latitude'], csi360_customers_df['Longitude'] = zip(*csi360_customers_df['zip'].apply(get_lat_long))

# Fill NaN values in 'Latitude' and 'Longitude' with empty strings
csi360_customers_df['Latitude'] = csi360_customers_df['Latitude'].fillna("")
csi360_customers_df['Longitude'] = csi360_customers_df['Longitude'].fillna("")

# Print the first few rows to verify
print(csi360_customers_df.head())


                            company           contact state Last Contact  \
0  Berkeley County Sheriff's Office  Lt. Geno Alterio    SC    1/17/2023   
1     Blytheville Police Department   Vanessa Stewart    AZ   10/27/2023   
2                      FEMA - Idaho      Keith Richey   NaN          NaN   
3  Franklin County Sheriff's Office      Rhonda Coyne    NC    8/22/2024   
4     Grant Parish Sheriff's Office     Cade Fletcher    LA     2/1/2023   

        Status    zip   Latitude  Longitude  
0   Custy-5/20  29461  33.146521 -79.986407  
1  Custy-10/20  72315  35.893377 -89.906631  
2          NaN                               
3   Custy-3/24  27549  36.073028 -78.247615  
4   Custy-8/22  71417  31.532732 -92.638779  


## Geocode Colorado Fire Deptartment Addresses


In [6]:
# Function to get latitude and longitude from address
# Set rate limit to 25 requests per second
# Google geocoding API allows around 50 queries per second, I've adjusted to 25 to be safe
RATE_LIMIT = 25  # requests per second
TIME_PERIOD = 1  # time period in seconds

# Function to get latitude and longitude from address with rate limiter
@sleep_and_retry
@limits(calls=RATE_LIMIT, period=TIME_PERIOD)
def get_lat_long(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return None, None

In [7]:
# Combine address fields and get latitude/longitude
lefd3_df['Full_Address'] = lefd3_df['addr1'] + ', ' + lefd3_df['city'] + ', ' + lefd3_df['state'] + ' ' + lefd3_df['zip'].astype(str)
lefd3_df['Latitude'], lefd3_df['Longitude'] = zip(*lefd3_df['Full_Address'].apply(get_lat_long))
print(lefd3_df.head())

                          agency_name              agency_type  \
0            San Antonio Park Rangers     Special jurisdiction   
1   College Station Police Department  Local police department   
2         Kingsport Police Department  Local police department   
3  Dorchester County Sheriff's Office         Sheriff's office   
4          Edinburg Police Department  Local police department   

   sworn_active_persnl hq_addr1 hq_addr2 hq_city hq_state hq_zip  \
0                  112                                             
1                  112                                             
2                  111                                             
3                  111                                             
4                  110                                             

                          addr1                 addr2  ...      county    zip  \
0   600 Hemisfair Park  No. 337                        ...       Bexar  78205   
1             2611 Texas Avenue 

In [8]:
# Save the dataframes to a new CSV file
csi360_customers_df.to_csv('resources/output/csi360_custy_latlong.csv', index=False)
lefd3_df.to_csv('resources/output/lefd3_latlong.csv', index=False)

## Use Haversine Formula to calculate distances between latitude and longitude pairs 
### Source: https://stackoverflow.com/questions/29545704/fast-haversine-approximation-python-pandas

In [9]:
# Load the CSV files into dataframes
csi360_customers_df = pd.read_csv('resources/output/csi360_custy_latlong.csv')
lefd3_latlong_df = pd.read_csv('resources/output/lefd3_latlong.csv')

print(csi360_customers_df.head(2), lefd3_latlong_df.head(2))

                            company           contact state Last Contact  \
0  Berkeley County Sheriff's Office  Lt. Geno Alterio    SC    1/17/2023   
1     Blytheville Police Department   Vanessa Stewart    AZ   10/27/2023   

        Status      zip   Latitude  Longitude  
0   Custy-5/20  29461.0  33.146521 -79.986407  
1  Custy-10/20  72315.0  35.893377 -89.906631                            agency_name              agency_type  \
0           San Antonio Park Rangers     Special jurisdiction   
1  College Station Police Department  Local police department   

   sworn_active_persnl hq_addr1 hq_addr2 hq_city hq_state hq_zip  \
0                  112                                             
1                  112                                             

                         addr1 addr2  ...  county    zip hq_ph hq_fax  \
0  600 Hemisfair Park  No. 337        ...   Bexar  78205                
1            2611 Texas Avenue        ...  Brazos  77842                

  org_

In [12]:
# Haversine formula to calculate the distance between two lat/lon pairs
def haversine(lat1, lon1, lat2, lon2):
    R = 3958.8  # Earth radius in miles
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Function to filter agencies within 100 miles AND in the same state using Haversine formula
def filter_agencies_within_100_miles_and_same_state(lefd3_latlong_df, csi360_customers_df):
    results = [] # Initialize an empty list to store results

 # Iterate over each company in csi360_customers_df
    for _, company_row in csi360_customers_df.iterrows():
        company_name = company_row['company']
        company_coords = (company_row['Latitude'], company_row['Longitude'])
        company_state = company_row['state']
        
 # Check each agency in lefd3_latlong_df
        for _, agency_row in lefd3_latlong_df.iterrows():
            try:
                agency_coords = (agency_row['Latitude'], agency_row['Longitude'])
                agency_state = agency_row['state']                

                # First, check if the agency is in the same state
                if agency_state == company_state:
                    # If in the same state, then calculate the distance
                    distance = haversine(company_coords[0], company_coords[1], agency_coords[0], agency_coords[1])
                
                    # If distance is within 100 miles, store the result
                    if distance <= 100:
                        results.append({
                            'company': company_name,
                            **agency_row.to_dict()  # Include all columns from lefd_latlong_df
                        })
            except Exception as e:
                print(f"Error calculating distance for {agency_row['agency_name']}: {e}")
    
    # Convert results to a DataFrame
    results3_df = pd.DataFrame(results)

    # Print the first 5 results for verification
    print(results3_df.head(5))

    # Save the results to a CSV file
    results3_df.to_csv('resources/output/lefd3_100mi_csicusty.csv', index=False)
  
    return results3_df

# Use the function with the loaded data
filter_agencies_within_100_miles_and_same_state(lefd3_latlong_df, csi360_customers_df)

                            company                            agency_name  \
0  Berkeley County Sheriff's Office     Dorchester County Sheriff's Office   
1  Berkeley County Sheriff's Office             Florence Police Department   
2  Berkeley County Sheriff's Office               Sumter Police Department   
3  Berkeley County Sheriff's Office  South Carolina State Transport Police   
4  Berkeley County Sheriff's Office     Orangeburg County Sheriff's Office   

               agency_type  sworn_active_persnl hq_addr1 hq_addr2 hq_city  \
0         Sheriff's office                  111                             
1  Local police department                  109                             
2  Local police department                  107                             
3     Special jurisdiction                  103                             
4         Sheriff's office                   92                             

  hq_state hq_zip                addr1  ...      county    zip hq_ph

,company,agency_name,agency_type,sworn_active_persnl,hq_addr1,hq_addr2,hq_city,hq_state,hq_zip,addr1,...,county,zip,hq_ph,hq_fax,org_type,website,fips,Full_Address,Latitude,Longitude
0,Berkeley County Sheriff's Office,Dorchester County Sheriff's Office,Sheriff's office,111,,,,,,100 Sears St,...,Dorchester,29477,,,,,45035,"100 Sears St, Saint George, SC 29477",33.185167,-80.578193
1,Berkeley County Sheriff's Office,Florence Police Department,Local police department,109,,,,,,8100 Ewing Blvd.,...,Florence,29501,,,,,45041,"8100 Ewing Blvd., Florence, SC 29501",34.195433,-79.762563
2,Berkeley County Sheriff's Office,Sumter Police Department,Local police department,107,,,,,,107 E. Hampton Ave.,...,Sumter,29150,,,,,45085,"107 E. Hampton Ave., Sumter, SC 29150",33.921903,-80.339296
3,Berkeley County Sheriff's Office,South Carolina State Transport Police,Special jurisdiction,103,,,,,,10311 Wilson Blvd,...,Richland,29016,,,,,45079,"10311 Wilson Blvd, Blythewood, SC 29016",34.178294,-80.973790
4,Berkeley County Sheriff's Office,Orangeburg County Sheriff's Office,Sheriff's office,92,,,,,,1520 Ellis Ave,...,Orangeburg,29118,,,,,45075,"1520 Ellis Ave, Orangeburg, SC 29118",33.511662,-80.852165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1061,Yoakum Police Department,Travis County Emergency Services District No. 2,Career,64,203 E Pecan ST,,Pflugerville,TX,78660-1260,203 E Pecan ST,...,Travis,78660-1260,512-251-2801,512-990-1125,"Local (includes career, combination, and volun...",https://www.pflugervillefire.com,,"203 E Pecan ST, Pflugerville, TX 78660-1260",30.438950,-97.619530
1062,Yoakum Police Department,Travis County Emergency Services District No. ...,Career,104,15304 Pheasant LN STE 100,,Austin,TX,78734,,...,Travis,78734-0004,512-266-2533,512-266-7065,"Local (includes career, combination, and volun...",https://www.ltfr.org/,,", Austin, TX 78734-0004",30.366667,-97.948390
1063,Yoakum Police Department,Travis County ESD No. 12 (Manor Volunteer Fire...,Career,31,11200 Gregg LN,,Manor,TX,78653,,...,Travis,78653-0846,512-272-4502,512-428-5114,"Local (includes career, combination, and volun...",http://tcesd12.com/,,", Manor, TX 78653-0846",30.341065,-97.557677
1064,Yoakum Police Department,Universal City Fire Department,Career,20,2160 Universal City BLVD,,Universal City,TX,78148-3492,2150 Universal City BLVD,...,Bexar,78148-3492,210-659-0333,210-659-5574,"Local (includes career, combination, and volun...",,,"2150 Universal City BLVD, Universal City, TX 7...",29.556920,-98.307963
